In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import re

import os
from glob import glob
from tqdm import tqdm

import yaml
from yaml import dump
import uuid
import itertools

In [3]:
import warnings
warnings.simplefilter(action="ignore")

In [4]:
def get_paths(models_list):
    '''
    Finds all the paths to forecasts and experiments metadata (directories /forecast/ and /wf_result/)
    
    Returns list with paths to forecast files, dict with metadata and list of all the experiment names
    '''
    
    uuids = []
    model_names = []
    train_start_or_duration = []
    hyperparameters = []
    features = []
    n_models = []
    
    paths_to_predictions = []
    paths_to_info = []

    forecast_paths = []
    metadata_paths = []


    for model in models_list:
        paths_to_predictions += glob(f'/masters_diploma/forecast/{model}/research_task_*/{model}_*/')
        paths_to_info += glob(f'/masters_diploma/wf_result/{model}/research_task_*')

    print(len(paths_to_predictions))
    print(len(paths_to_info))
            
    for path2 in paths_to_info:   
#     for path2 in [max(paths_to_info, key=os.path.getctime)]:   # тільки для останнього експерименту
        metadata_paths.extend(glob(os.path.join(path2, '*.csv')))


    for path2 in paths_to_predictions:
#     for path2 in [max(paths_to_predictions, key=os.path.getctime)]:   # тільки для останнього експерименту
        prediction_paths = glob(os.path.join(path2, f'*.csv'))
        if len(prediction_paths) > 0:
            forecast_paths.append(prediction_paths)

    yaml_file_paths = [f.replace('.csv', '.yaml') for f in metadata_paths]

    metadata = {}
    experiment_names = []
    for file in yaml_file_paths: 
        with open(file, 'r') as f:
            res = yaml.safe_load(f)

        shorten_uuid = "-".join([res['unique_uuid'].split('-')[0], res['unique_uuid'].split('-')[-2]])
        dur = res['duration_training_history'] if 'duration_training_history' in res else res['train_start']

        metadata[shorten_uuid] = {
            "uuid": res['unique_uuid'],
            "model_name": res['model_name'],
            "duration/train_start": dur,
            "hyperparameters": res['model_hyperparameters'],
            "features": res['train_features']
        }

        experiment_names.append(f"{res['model_name']}-{shorten_uuid}")


    # for file in yaml_file_paths: 
    #     with open(file, 'r') as f:
    #         res = yaml.safe_load(f)
    #     uuids.append(res['unique_uuid'])
    #     model_names.append(res['model_name'])
    #     hyperparameters.append(res['model_hyperparameters'])
    #     features.append(res['train_features'])
    #     if 'duration_training_history' in res:
    #         train_start_or_duration.append(res['duration_training_history'])
    #     else:
    #         train_start_or_duration.append(res['train_start'])
     
    # shorten_uuids = ["-".join([n.split('-')[0], n.split('-')[-2]]) for n in uuids]
    # metadata_lst = list(zip(uuids, shorten_uuids, model_names, train_start_or_duration, hyperparameters, features, n_models))           
    
    # file_paths_splited = [metadata_paths[k].split('-') for k in range(len(metadata_paths))]
    # model_name = [file_paths_splited[k][-5].split('\\')[1] for k in range(len(metadata_paths))]
    # experiment_names = ["-".join([model_name[k], file_paths_splited[k][-2]]) for k in range(len(metadata_paths))]     
    

    return forecast_paths, metadata_paths, metadata, experiment_names

In [5]:
def make_forecasts_df():
    pass

    return forecasts

In [6]:
def add_fact(forecasts):
    pass

In [7]:
def get_stat_for_d1():
    pass

    return stat_d1

In [8]:
def get_best_models_per_hour():
    pass

    return best_models

In [9]:
path_to_files = '/masters_diploma/'
models_list = ['xgboost', 'random_forest', 'lightgbm']

paths, metadata_paths, metadata_dict, exp_names = get_paths(models_list)

for key, info in metadata_dict.items():




for i, path in enumerate(metadata_paths): 
    id_exp = path.split('/')[-2].split('_')[-1]
    print(path.split('/')[-2])
    exp_forecasts = paths[id_exp]
    metadata = [k for k in metadata_dict if k[i] == id_exp][0]
    exp_name = [k for k in exp_names if k[i] == id_exp][0]
    
    df_preds = make_forecasts_df(exp_forecasts, metadata, exp_name)
    stat, stat_per_h = get_stat_for_d1(df_preds, df_preds.shape[0], exp_name)
    best_models_df = get_best_models_per_hour(stat_per_h)



gen_stat_df = df.read_csv(os.path.join(path_to_files, 'statistics', 'general_statistics'))
gen_stat_df = pd.concat([gen_stat_df, stat]).drop_duplicates()

gen_stat_df.to_csv(os.path.join(path_to_files, 'statistics', 'general_statistics'))

164
3


In [10]:
paths

[['/masters_diploma/forecast/xgboost\\research_task_dcde6e0a-df0d-11ef-b7df-3417ebde98e4\\xgboost_dcde754a-df0d-11ef-92fb-3417ebde98e4\\forecast_d-0_xgboost_2025-01-01_(2025-01-01).csv',
  '/masters_diploma/forecast/xgboost\\research_task_dcde6e0a-df0d-11ef-b7df-3417ebde98e4\\xgboost_dcde754a-df0d-11ef-92fb-3417ebde98e4\\forecast_d-0_xgboost_2025-01-02_(2025-01-02).csv',
  '/masters_diploma/forecast/xgboost\\research_task_dcde6e0a-df0d-11ef-b7df-3417ebde98e4\\xgboost_dcde754a-df0d-11ef-92fb-3417ebde98e4\\forecast_d-0_xgboost_2025-01-03_(2025-01-03).csv',
  '/masters_diploma/forecast/xgboost\\research_task_dcde6e0a-df0d-11ef-b7df-3417ebde98e4\\xgboost_dcde754a-df0d-11ef-92fb-3417ebde98e4\\forecast_d-0_xgboost_2025-01-04_(2025-01-04).csv',
  '/masters_diploma/forecast/xgboost\\research_task_dcde6e0a-df0d-11ef-b7df-3417ebde98e4\\xgboost_dcde754a-df0d-11ef-92fb-3417ebde98e4\\forecast_d-0_xgboost_2025-01-05_(2025-01-05).csv',
  '/masters_diploma/forecast/xgboost\\research_task_dcde6e0a-df0d